# IBM Data Science Professional Certificate
## Capstone Project

In [1]:
# importing relevant libraries

import pandas as pd

import requests # library to handle requests
import numpy as np
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe


from geopy.geocoders import Nominatim
from IPython.display import JSON

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
# Pretty print
from pprint import pprint

print('Libraries imported.')

Libraries imported.


In [2]:
data = pd.read_csv('https://raw.githubusercontent.com/nessieee/Coursera_Capstone/master/VA_PostalCodes.csv', sep=";", header=None)

In [3]:
data.head()

,0,1,2,3,4,5
0,CA,V5K,Vancouver (North Hastings-Sunrise),British Columbia,49.2807,-123.0397
1,CA,V5L,Vancouver (North Grandview-Woodlands),British Columbia,49.2795,-123.0667
2,CA,V5M,Vancouver (South Hastings-Sunrise / North Renf...,British Columbia,49.2600,-123.0398
3,CA,V5N,Vancouver (South Grandview-Woodlands / NE Kens...,British Columbia,49.2551,-123.0667
4,CA,V5P,Vancouver (SE Kensington / Victoria-Fraserview),British Columbia,49.2220,-123.0683


In [4]:
data.dtypes

0     object
1     object
2     object
3     object
4    float64
5    float64
dtype: object

In [5]:
data.columns= ["CountryCode", "FSA", "Neighbourhoods", "Province", "Latitude","Longitude"]
data.head()

,CountryCode,FSA,Neighbourhoods,Province,Latitude,Longitude
0,CA,V5K,Vancouver (North Hastings-Sunrise),British Columbia,49.2807,-123.0397
1,CA,V5L,Vancouver (North Grandview-Woodlands),British Columbia,49.2795,-123.0667
2,CA,V5M,Vancouver (South Hastings-Sunrise / North Renf...,British Columbia,49.2600,-123.0398
3,CA,V5N,Vancouver (South Grandview-Woodlands / NE Kens...,British Columbia,49.2551,-123.0667
4,CA,V5P,Vancouver (SE Kensington / Victoria-Fraserview),British Columbia,49.2220,-123.0683


In [6]:
data.shape

(44, 6)

In [7]:
address = 'Vancouver, British Colombia'

geolocator = Nominatim(user_agent="vancouver_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Vancouver are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Vancouver are 49.2608724, -123.1139529.


In [8]:
# create map of hyderabad using latitude and longitude values
map_vanc = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighbourhood in zip(data['Latitude'], data['Longitude'], data['Neighbourhoods']):
    label = '{}'.format(neighbourhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_vanc)  
    
map_vanc

In [9]:
CLIENT_ID = 'IZFNLD5OADUQ0VKMNCGDB4PAKRTAKFIZTTXIN2Y2R534Q4SA' # your Foursquare ID
CLIENT_SECRET = 'KNFWLYA5SQAHVSRBOKMXJ4B4HEGJTOPO4ACGN3NEI30VRH4E' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: IZFNLD5OADUQ0VKMNCGDB4PAKRTAKFIZTTXIN2Y2R534Q4SA
CLIENT_SECRET:KNFWLYA5SQAHVSRBOKMXJ4B4HEGJTOPO4ACGN3NEI30VRH4E


In [10]:
latitude = data['Latitude'][14]
longitude = data['Longitude'][14]
radius = 1500
LIMIT = 100

In [11]:
from IPython.display import JSON

# define URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT)

# send GET request and get recommended venues
results = requests.get(url).json()
print('There are {} recommended venues.'.format(len(results['response']['groups'][0]['items'])))

JSON(results)

There are 100 recommended venues.


<IPython.core.display.JSON object>

In [12]:
# Part adapted from example exercise
def get_category_type(row):
    'A function that extracts the category of the venue in FourSquare JSON'
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']


# Get relevant part of FourSquare's explore call JSON
items = results['response']['groups'][0]['items']

dataframe = json_normalize(items) # flatten JSON

# Filter columns
filtered_columns = ['venue.name', 'venue.categories'] + [col for col in dataframe.columns if col.startswith('venue.location.')] + ['venue.id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# Filter the category for each row
dataframe_filtered['venue.categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# Clean columns
dataframe_filtered.columns = [col.split('.')[-1] for col in dataframe_filtered.columns]

dataframe_filtered.head(15)

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Queen Elizabeth Theatre,Theater,649 Cambie St,CA,Vancouver,Canada,at W Georgia St,180,"[649 Cambie St (at W Georgia St), Vancouver BC...","[{'label': 'display', 'lat': 49.28022895029647...",49.280229,-123.112730,NaN,V6B 2P1,BC,4b23212bf964a520a75324e3
1,BC Place,Stadium,777 Pacific Blvd.,CA,Vancouver,Canada,at Expo Blvd.,243,"[777 Pacific Blvd. (at Expo Blvd.), Vancouver ...",NaN,49.276886,-123.112276,NaN,V6B 4Y8,BC,4aa6f277f964a520964b20e3
2,L'Hermitage,Hotel,788 Richard St,CA,Vancouver,Canada,NaN,299,"[788 Richard St, Vancouver BC V6B 3A4, Canada]","[{'label': 'display', 'lat': 49.2801392339119,...",49.280139,-123.117480,NaN,V6B 3A4,BC,4b7235fef964a52010742de3
3,JJ Bean,Coffee Shop,740 Hamilton St.,CA,Vancouver,Canada,btwn Robson & Georgia,113,"[740 Hamilton St. (btwn Robson & Georgia), Van...","[{'label': 'display', 'lat': 49.27938179808702...",49.279382,-123.115181,NaN,V6B 2R5,BC,4b75918bf964a520d4142ee3
4,Medina Café,Breakfast Spot,780 Richards St,CA,Vancouver,Canada,btwn W Georgia & Robson,291,"[780 Richards St (btwn W Georgia & Robson), Va...","[{'label': 'display', 'lat': 49.28056540642322...",49.280565,-123.116859,NaN,V6B 2L3,BC,4aa7dce2f964a520ad4d20e3
5,Chambar,Belgian Restaurant,568 Beatty St,CA,Vancouver,Canada,btwn W Pender & Dunsmuir,304,"[568 Beatty St (btwn W Pender & Dunsmuir), Van...","[{'label': 'display', 'lat': 49.27992541607816...",49.279925,-123.110078,NaN,V6B 2L3,BC,4aa8005bf964a520a24e20e3
6,Fanny Bay Oyster Bar,Seafood Restaurant,762 Cambie St,CA,Vancouver,Canada,NaN,43,"[762 Cambie St, Vancouver BC V6B 2P2, Canada]","[{'label': 'display', 'lat': 49.27860106769942...",49.278601,-123.114411,NaN,V6B 2P2,BC,5790281f498e75e0454bf0a9
7,Homer St. Cafe and Bar,Restaurant,898 Homer St.,CA,Vancouver,Canada,at Smithe St.,326,"[898 Homer St. (at Smithe St.), Vancouver BC V...","[{'label': 'display', 'lat': 49.27859106047440...",49.278591,-123.118385,NaN,V6B 2W5,BC,51a7a349454a2b1f0e2449f8
8,The Dirty Apron Cooking School,Trade School,540 Beatty St,CA,Vancouver,Canada,btwn Dunsmuir & Pender,389,"[540 Beatty St (btwn Dunsmuir & Pender), Vanco...","[{'label': 'display', 'lat': 49.28069252156226...",49.280693,-123.109393,NaN,V6B 2L3,BC,4ac00474f964a520779320e3
9,Rogers Arena,Hockey Arena,800 Griffiths Way,CA,Vancouver,Canada,at Abbott St,388,"[800 Griffiths Way (at Abbott St), Vancouver B...","[{'label': 'display', 'lat': 49.27792296883324...",49.277923,-123.108723,NaN,V6B 6G1,BC,4aa6f23df964a520954b20e3


In [13]:
radius = 2000
LIMIT = 100

venues = []

    
for lat, long, neighbourhood in zip(data['Latitude'], data['Longitude'], data['Neighbourhoods']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighbourhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [14]:
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighbourhoods', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head(20)

(3325, 7)


,Neighbourhoods,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Vancouver (North Hastings-Sunrise),49.2807,-123.0397,The Fair at the PNE,49.282971,-123.042109,Fair
1,Vancouver (North Hastings-Sunrise),49.2807,-123.0397,Wooden Roller Coaster,49.281744,-123.035128,Theme Park Ride / Attraction
2,Vancouver (North Hastings-Sunrise),49.2807,-123.0397,Livestock Barns,49.284037,-123.039278,Farm
3,Vancouver (North Hastings-Sunrise),49.2807,-123.0397,Tamam Fine Palestinian Cuisine,49.281070,-123.051438,Middle Eastern Restaurant
4,Vancouver (North Hastings-Sunrise),49.2807,-123.0397,New Brighton Park,49.289614,-123.038484,Park
5,Vancouver (North Hastings-Sunrise),49.2807,-123.0397,What's Up? Hot Dog!,49.281252,-123.055060,Hot Dog Joint
6,Vancouver (North Hastings-Sunrise),49.2807,-123.0397,Bao Chau,49.281232,-123.048971,Vietnamese Restaurant
7,Vancouver (North Hastings-Sunrise),49.2807,-123.0397,Tacofino Commissary,49.281281,-123.057509,Mexican Restaurant
8,Vancouver (North Hastings-Sunrise),49.2807,-123.0397,Platform 7 Coffee,49.281269,-123.057445,Coffee Shop
9,Vancouver (North Hastings-Sunrise),49.2807,-123.0397,The Red Wagon,49.281155,-123.057997,Breakfast Spot


In [15]:
venues_df.groupby(["Neighbourhoods"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighbourhoods,,,,,,
North Vancouver East Central,10,10,10,10,10,10
North Vancouver Inner East,2,2,2,2,2,2
North Vancouver North Central,10,10,10,10,10,10
North Vancouver Northwest,33,33,33,33,33,33
North Vancouver Northwest Central,18,18,18,18,18,18
North Vancouver Outer East,10,10,10,10,10,10
North Vancouver South Central,100,100,100,100,100,100
North Vancouver Southwest,100,100,100,100,100,100
North Vancouver Southwest Central,100,100,100,100,100,100


In [16]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 249 uniques categories.


In [17]:
# Filter subdistrict
lowvenue_neighbourhood = venues_df.groupby('Neighbourhoods').VenueName.count() < 10
lowvenue_neighbourhood = list(lowvenue_neighbourhood[lowvenue_neighbourhood].index)

# duplicate df
van_venues = venues_df

# Exclude the subdistricts
for i in lowvenue_neighbourhood:
    van_venues = van_venues[van_venues.Neighbourhoods != i]

In [18]:

van_venues.groupby('Neighbourhoods').count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighbourhoods,,,,,,
North Vancouver East Central,10,10,10,10,10,10
North Vancouver North Central,10,10,10,10,10,10
North Vancouver Northwest,33,33,33,33,33,33
North Vancouver Northwest Central,18,18,18,18,18,18
North Vancouver Outer East,10,10,10,10,10,10
North Vancouver South Central,100,100,100,100,100,100
North Vancouver Southwest,100,100,100,100,100,100
North Vancouver Southwest Central,100,100,100,100,100,100
Vancouver (Bentall Centre),100,100,100,100,100,100


In [19]:
van_venues['VenueCategory'].unique()[:50]

array(['Fair', 'Theme Park Ride / Attraction', 'Farm',
       'Middle Eastern Restaurant', 'Park', 'Hot Dog Joint',
       'Vietnamese Restaurant', 'Mexican Restaurant', 'Coffee Shop',
       'Breakfast Spot', 'Grocery Store', 'Indian Restaurant',
       'Food Truck', 'Hockey Arena', 'Café', 'Bakery', 'Theme Park',
       'Soccer Field', 'Hobby Shop', 'Dessert Shop', 'Brewery',
       'Sushi Restaurant', 'Racetrack', 'Southern / Soul Food Restaurant',
       'Motorcycle Shop', 'French Restaurant', 'Beer Garden', 'Theater',
       'Amphitheater', 'Deli / Bodega', 'Latin American Restaurant',
       'Thai Restaurant', 'Greek Restaurant', 'Japanese Restaurant',
       'Event Space', 'Seafood Restaurant', 'BBQ Joint',
       'Fried Chicken Joint', 'Chinese Restaurant', 'Liquor Store',
       'Asian Restaurant', 'Bank', 'Thrift / Vintage Store', 'Restaurant',
       'Pub', 'Sandwich Place', 'Convenience Store',
       'American Restaurant', 'Pool', 'Beach'], dtype=object)

## Analysis
### Applying one-hot encoding to the dataframe.

In [20]:
# one hot encoding
van_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighbourhood column back to dataframe
van_onehot['Neighbourhoods'] = venues_df['Neighbourhoods'] 

# move neighbourhood column to the first column
fixed_columns = [van_onehot.columns[-1]] + list(van_onehot.columns[:-1])
van_onehot = van_onehot[fixed_columns]

# checking the shape
print(van_onehot.shape)
van_onehot.head()

(3325, 250)


,Neighbourhoods,Accessories Store,American Restaurant,Amphitheater,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,...,Tree,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Volleyball Court,Water Park,Waterfront,Wine Shop,Women's Store,Yoga Studio
0,Vancouver (North Hastings-Sunrise),0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Vancouver (North Hastings-Sunrise),0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Vancouver (North Hastings-Sunrise),0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Vancouver (North Hastings-Sunrise),0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Vancouver (North Hastings-Sunrise),0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [21]:
# grouping the encoded dataframe
van_grouped = van_onehot.groupby(["Neighbourhoods"]).mean().reset_index()

print(van_grouped.shape)
van_grouped


(43, 250)


,Neighbourhoods,Accessories Store,American Restaurant,Amphitheater,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,...,Tree,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Volleyball Court,Water Park,Waterfront,Wine Shop,Women's Store,Yoga Studio
0,North Vancouver East Central,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.00,0.000000
1,North Vancouver Inner East,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.00,0.000000
2,North Vancouver North Central,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.00,0.000000
3,North Vancouver Northwest,0.000000,0.000000,0.000000,0.00,0.030303,0.000000,0.000000,0.00,0.00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.00,0.000000
4,North Vancouver Northwest Central,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.00,0.000000
5,North Vancouver Outer East,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.00,0.000000
6,North Vancouver South Central,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.010000,0.01,0.00,...,0.000000,0.010000,0.000000,0.010000,0.000000,0.00,0.00,0.00,0.00,0.000000
7,North Vancouver Southwest,0.000000,0.010000,0.000000,0.00,0.000000,0.000000,0.010000,0.00,0.00,...,0.000000,0.010000,0.000000,0.010000,0.000000,0.00,0.00,0.01,0.01,0.000000
8,North Vancouver Southwest Central,0.000000,0.010000,0.000000,0.00,0.000000,0.000000,0.010000,0.00,0.00,...,0.000000,0.020000,0.000000,0.000000,0.000000,0.00,0.00,0.01,0.00,0.000000
9,Vancouver (Bentall Centre),0.010000,0.020000,0.000000,0.01,0.010000,0.000000,0.000000,0.00,0.00,...,0.000000,0.010000,0.000000,0.010000,0.000000,0.00,0.01,0.00,0.00,0.010000


In [22]:
# printing 5 most common venues for every neighbourhood

num_top_venues = 5

for hood in van_grouped['Neighbourhoods']:
    print("----"+hood+"----")
    temp = van_grouped[van_grouped['Neighbourhoods'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----North Vancouver East Central----
            venue  freq
0           Trail   0.3
1            Lake   0.2
2            Park   0.2
3     Coffee Shop   0.1
4  Scenic Lookout   0.1


----North Vancouver Inner East----
                venue  freq
0            Ski Area   0.5
1               Trail   0.5
2   Accessories Store   0.0
3        Outlet Store   0.0
4  Persian Restaurant   0.0


----North Vancouver North Central----
            venue  freq
0           Trail   0.3
1            Park   0.2
2            Lake   0.1
3  Scenic Lookout   0.1
4     Coffee Shop   0.1


----North Vancouver Northwest----
            venue  freq
0           Trail  0.27
1  Scenic Lookout  0.12
2   Ski Chairlift  0.09
3      Ski Chalet  0.06
4     Coffee Shop  0.06


----North Vancouver Northwest Central----
                  venue  freq
0                  Park  0.17
1                 Trail  0.17
2           Coffee Shop  0.11
3      Business Service  0.06
4  Gym / Fitness Center  0.06


----North Vancouver Oute

                   venue  freq
0            Coffee Shop  0.09
1                   Café  0.06
2  Vietnamese Restaurant  0.05
3       Sushi Restaurant  0.04
4                 Bakery  0.04


----West Vancouver North----
            venue  freq
0            Lake  0.29
1       Ski Lodge  0.14
2        Ski Area  0.14
3  Scenic Lookout  0.14
4           Trail  0.14


----West Vancouver South----
              venue  freq
0       Coffee Shop  0.12
1            Bakery  0.08
2         Gastropub  0.04
3  Sushi Restaurant  0.04
4          Building  0.04


----West Vancouver Southeast----
            venue  freq
0     Coffee Shop  0.11
1            Bank  0.04
2  Clothing Store  0.04
3           Hotel  0.04
4   Grocery Store  0.04


----West Vancouver West----
              venue  freq
0       Coffee Shop  0.12
1     Boat or Ferry  0.10
2   Harbor / Marina  0.10
3             Trail  0.07
4  Sushi Restaurant  0.05




In [23]:
# returning most common venues

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [24]:
# Creating a new dataframe that will display the top 10 venues for each neighbourhood
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhoods']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhoods'] = van_grouped['Neighbourhoods']

for ind in np.arange(van_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(van_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

,Neighbourhoods,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North Vancouver East Central,Trail,Park,Lake,Mountain,Coffee Shop,Scenic Lookout,French Restaurant,Fried Chicken Joint,Food Truck,Filipino Restaurant
1,North Vancouver Inner East,Ski Area,Trail,Yoga Studio,Food & Drink Shop,Financial or Legal Service,Fish & Chips Shop,Flower Shop,Food,Food Court,Field
2,North Vancouver North Central,Trail,Park,Scenic Lookout,Coffee Shop,Mountain,Lake,Convenience Store,Food Court,Fish & Chips Shop,Flower Shop
3,North Vancouver Northwest,Trail,Scenic Lookout,Ski Chairlift,Coffee Shop,Mountain,Ski Chalet,Restaurant,Fast Food Restaurant,Dessert Shop,Sporting Goods Shop
4,North Vancouver Northwest Central,Trail,Park,Coffee Shop,Convenience Store,Chinese Restaurant,Shopping Mall,Grocery Store,Bar,Gym / Fitness Center,Sandwich Place


In [25]:
len(van_grouped[van_grouped["Pub"] > 0])

20

In [26]:
van_place = van_grouped[["Neighbourhoods","Pub"]]
van_place.head()

,Neighbourhoods,Pub
0,North Vancouver East Central,0.0
1,North Vancouver Inner East,0.0
2,North Vancouver North Central,0.0
3,North Vancouver Northwest,0.0
4,North Vancouver Northwest Central,0.0


## Cluster Neighbourhoods

In [27]:
# set number of clusters
kclusters = 5

van_clustering = van_grouped.drop(["Neighbourhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(van_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([2, 1, 2, 4, 2, 4, 0, 0, 0, 3])

In [28]:
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

van_merged = data

# drop NA from the excluded subdistrict
for i in lowvenue_neighbourhood:
    van_merged = van_merged[van_merged.Neighbourhoods != i]

# merge df to add latitude/longitude for each subdistrict
van_merged = van_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhoods'), on='Neighbourhoods')

# Shift label to start from index 1
van_merged['Cluster Labels'] = van_merged['Cluster Labels'] + 1

van_merged

,CountryCode,FSA,Neighbourhoods,Province,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,CA,V5K,Vancouver (North Hastings-Sunrise),British Columbia,49.2807,-123.0397,1.0,Vietnamese Restaurant,Café,Coffee Shop,Park,Bakery,French Restaurant,Chinese Restaurant,Pub,Brewery,Sushi Restaurant
1,CA,V5L,Vancouver (North Grandview-Woodlands),British Columbia,49.2795,-123.0667,1.0,Coffee Shop,Brewery,Café,Pizza Place,Sushi Restaurant,Italian Restaurant,Indian Restaurant,Grocery Store,Ice Cream Shop,Vegetarian / Vegan Restaurant
2,CA,V5M,Vancouver (South Hastings-Sunrise / North Renf...,British Columbia,49.2600,-123.0398,1.0,Coffee Shop,Park,Bus Stop,Pizza Place,Grocery Store,Convenience Store,Burger Joint,Bus Station,Pet Store,Metro Station
3,CA,V5N,Vancouver (South Grandview-Woodlands / NE Kens...,British Columbia,49.2551,-123.0667,1.0,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Coffee Shop,Pizza Place,Italian Restaurant,Café,Mexican Restaurant,Indian Restaurant,Japanese Restaurant,Bakery
4,CA,V5P,Vancouver (SE Kensington / Victoria-Fraserview),British Columbia,49.2220,-123.0683,1.0,Chinese Restaurant,Vietnamese Restaurant,Japanese Restaurant,Coffee Shop,Sandwich Place,Asian Restaurant,Gas Station,Bank,Bakery,Cantonese Restaurant
5,CA,V5R,Vancouver (South Renfrew-Collingwood),British Columbia,49.2397,-123.0407,1.0,Chinese Restaurant,Coffee Shop,Vietnamese Restaurant,Asian Restaurant,Pizza Place,Japanese Restaurant,Park,Café,Sushi Restaurant,Convenience Store
6,CA,V5S,Vancouver (Killarney),British Columbia,49.2175,-123.0380,1.0,Coffee Shop,Park,Golf Course,Bus Stop,Convenience Store,Sandwich Place,Chinese Restaurant,Café,Burger Joint,Bubble Tea Shop
7,CA,V5T,Vancouver (East Mount Pleasant),British Columbia,49.2620,-123.0923,1.0,Brewery,Coffee Shop,Bakery,Vietnamese Restaurant,Sushi Restaurant,Indian Restaurant,Café,Vegetarian / Vegan Restaurant,Italian Restaurant,Yoga Studio
8,CA,V5V,Vancouver (West Kensington / NE Riley Park-Lit...,British Columbia,49.2480,-123.0913,1.0,Vietnamese Restaurant,Coffee Shop,Park,Café,Vegetarian / Vegan Restaurant,Sushi Restaurant,Bakery,Ice Cream Shop,Seafood Restaurant,Indian Restaurant
9,CA,V5W,Vancouver (SE Riley Park-Little Mountain / SW ...,British Columbia,49.2327,-123.0917,1.0,Chinese Restaurant,Coffee Shop,Vietnamese Restaurant,Café,Japanese Restaurant,Park,Pizza Place,Garden,Farmers Market,Tea Room


In [29]:
van_merged.dtypes

CountryCode                object
FSA                        object
Neighbourhoods             object
Province                   object
Latitude                  float64
Longitude                 float64
Cluster Labels            float64
1st Most Common Venue      object
2nd Most Common Venue      object
3rd Most Common Venue      object
4th Most Common Venue      object
5th Most Common Venue      object
6th Most Common Venue      object
7th Most Common Venue      object
8th Most Common Venue      object
9th Most Common Venue      object
10th Most Common Venue     object
dtype: object

<b> Comment: <b> Cluster Labels is a float. For the clustering to work properly, it needs to be modified to an integer. To do so, I will first check if there is a NaN value in the dataframe. After eliminating it, Cluster Labels will be changed to integer data type.

In [30]:
van_merged.isnull()

,CountryCode,FSA,Neighbourhoods,Province,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
5,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
6,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
7,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
8,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
9,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [31]:
van_merged = van_merged[van_merged['Cluster Labels'].notna()]

<b> Comment:<b> As seen above, the analysis did not provide any results for most common venues for FSA = V6T, Neighbourhood = 	Vancouver (UBC). This postal code relates to the University of British Columbia - it should be excluded from the analysis.

In [32]:
van_merged['Cluster Labels'] =van_merged['Cluster Labels'].astype(int)

C:\Users\48500\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [33]:
van_merged.reset_index()
van_merged.dtypes

CountryCode                object
FSA                        object
Neighbourhoods             object
Province                   object
Latitude                  float64
Longitude                 float64
Cluster Labels              int32
1st Most Common Venue      object
2nd Most Common Venue      object
3rd Most Common Venue      object
4th Most Common Venue      object
5th Most Common Venue      object
6th Most Common Venue      object
7th Most Common Venue      object
8th Most Common Venue      object
9th Most Common Venue      object
10th Most Common Venue     object
dtype: object

In [34]:
van_merged

,CountryCode,FSA,Neighbourhoods,Province,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,CA,V5K,Vancouver (North Hastings-Sunrise),British Columbia,49.2807,-123.0397,1,Vietnamese Restaurant,Café,Coffee Shop,Park,Bakery,French Restaurant,Chinese Restaurant,Pub,Brewery,Sushi Restaurant
1,CA,V5L,Vancouver (North Grandview-Woodlands),British Columbia,49.2795,-123.0667,1,Coffee Shop,Brewery,Café,Pizza Place,Sushi Restaurant,Italian Restaurant,Indian Restaurant,Grocery Store,Ice Cream Shop,Vegetarian / Vegan Restaurant
2,CA,V5M,Vancouver (South Hastings-Sunrise / North Renf...,British Columbia,49.2600,-123.0398,1,Coffee Shop,Park,Bus Stop,Pizza Place,Grocery Store,Convenience Store,Burger Joint,Bus Station,Pet Store,Metro Station
3,CA,V5N,Vancouver (South Grandview-Woodlands / NE Kens...,British Columbia,49.2551,-123.0667,1,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Coffee Shop,Pizza Place,Italian Restaurant,Café,Mexican Restaurant,Indian Restaurant,Japanese Restaurant,Bakery
4,CA,V5P,Vancouver (SE Kensington / Victoria-Fraserview),British Columbia,49.2220,-123.0683,1,Chinese Restaurant,Vietnamese Restaurant,Japanese Restaurant,Coffee Shop,Sandwich Place,Asian Restaurant,Gas Station,Bank,Bakery,Cantonese Restaurant
5,CA,V5R,Vancouver (South Renfrew-Collingwood),British Columbia,49.2397,-123.0407,1,Chinese Restaurant,Coffee Shop,Vietnamese Restaurant,Asian Restaurant,Pizza Place,Japanese Restaurant,Park,Café,Sushi Restaurant,Convenience Store
6,CA,V5S,Vancouver (Killarney),British Columbia,49.2175,-123.0380,1,Coffee Shop,Park,Golf Course,Bus Stop,Convenience Store,Sandwich Place,Chinese Restaurant,Café,Burger Joint,Bubble Tea Shop
7,CA,V5T,Vancouver (East Mount Pleasant),British Columbia,49.2620,-123.0923,1,Brewery,Coffee Shop,Bakery,Vietnamese Restaurant,Sushi Restaurant,Indian Restaurant,Café,Vegetarian / Vegan Restaurant,Italian Restaurant,Yoga Studio
8,CA,V5V,Vancouver (West Kensington / NE Riley Park-Lit...,British Columbia,49.2480,-123.0913,1,Vietnamese Restaurant,Coffee Shop,Park,Café,Vegetarian / Vegan Restaurant,Sushi Restaurant,Bakery,Ice Cream Shop,Seafood Restaurant,Indian Restaurant
9,CA,V5W,Vancouver (SE Riley Park-Little Mountain / SW ...,British Columbia,49.2327,-123.0917,1,Chinese Restaurant,Coffee Shop,Vietnamese Restaurant,Café,Japanese Restaurant,Park,Pizza Place,Garden,Farmers Market,Tea Room


In [35]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(van_merged['Latitude'], van_merged['Longitude'], van_merged['Neighbourhoods'], van_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [36]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

## Analyzing clusters

### Cluster 1

In [37]:

van_merged.loc[van_merged['Cluster Labels'] == 1, van_merged.columns[[2] + list(range(4, van_merged.shape[1]))]]

,Neighbourhoods,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Vancouver (North Hastings-Sunrise),49.2807,-123.0397,1,Vietnamese Restaurant,Café,Coffee Shop,Park,Bakery,French Restaurant,Chinese Restaurant,Pub,Brewery,Sushi Restaurant
1,Vancouver (North Grandview-Woodlands),49.2795,-123.0667,1,Coffee Shop,Brewery,Café,Pizza Place,Sushi Restaurant,Italian Restaurant,Indian Restaurant,Grocery Store,Ice Cream Shop,Vegetarian / Vegan Restaurant
2,Vancouver (South Hastings-Sunrise / North Renf...,49.2600,-123.0398,1,Coffee Shop,Park,Bus Stop,Pizza Place,Grocery Store,Convenience Store,Burger Joint,Bus Station,Pet Store,Metro Station
3,Vancouver (South Grandview-Woodlands / NE Kens...,49.2551,-123.0667,1,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Coffee Shop,Pizza Place,Italian Restaurant,Café,Mexican Restaurant,Indian Restaurant,Japanese Restaurant,Bakery
4,Vancouver (SE Kensington / Victoria-Fraserview),49.2220,-123.0683,1,Chinese Restaurant,Vietnamese Restaurant,Japanese Restaurant,Coffee Shop,Sandwich Place,Asian Restaurant,Gas Station,Bank,Bakery,Cantonese Restaurant
5,Vancouver (South Renfrew-Collingwood),49.2397,-123.0407,1,Chinese Restaurant,Coffee Shop,Vietnamese Restaurant,Asian Restaurant,Pizza Place,Japanese Restaurant,Park,Café,Sushi Restaurant,Convenience Store
6,Vancouver (Killarney),49.2175,-123.0380,1,Coffee Shop,Park,Golf Course,Bus Stop,Convenience Store,Sandwich Place,Chinese Restaurant,Café,Burger Joint,Bubble Tea Shop
7,Vancouver (East Mount Pleasant),49.2620,-123.0923,1,Brewery,Coffee Shop,Bakery,Vietnamese Restaurant,Sushi Restaurant,Indian Restaurant,Café,Vegetarian / Vegan Restaurant,Italian Restaurant,Yoga Studio
8,Vancouver (West Kensington / NE Riley Park-Lit...,49.2480,-123.0913,1,Vietnamese Restaurant,Coffee Shop,Park,Café,Vegetarian / Vegan Restaurant,Sushi Restaurant,Bakery,Ice Cream Shop,Seafood Restaurant,Indian Restaurant
9,Vancouver (SE Riley Park-Little Mountain / SW ...,49.2327,-123.0917,1,Chinese Restaurant,Coffee Shop,Vietnamese Restaurant,Café,Japanese Restaurant,Park,Pizza Place,Garden,Farmers Market,Tea Room


## Cluster 2

In [38]:
van_merged.loc[van_merged['Cluster Labels'] == 2, van_merged.columns[[2] + list(range(4, van_merged.shape[1]))]]

,Neighbourhoods,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue


## Cluster 3

In [39]:
van_merged.loc[van_merged['Cluster Labels'] == 3, van_merged.columns[[2] + list(range(4, van_merged.shape[1]))]]

,Neighbourhoods,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
31,North Vancouver East Central,49.3622,-123.0178,3,Trail,Park,Lake,Mountain,Coffee Shop,Scenic Lookout,French Restaurant,Fried Chicken Joint,Food Truck,Filipino Restaurant
32,North Vancouver North Central,49.3597,-123.0377,3,Trail,Park,Scenic Lookout,Coffee Shop,Mountain,Lake,Convenience Store,Food Court,Fish & Chips Shop,Flower Shop
35,North Vancouver Northwest Central,49.3500,-123.0679,3,Trail,Park,Coffee Shop,Convenience Store,Chinese Restaurant,Shopping Mall,Grocery Store,Bar,Gym / Fitness Center,Sandwich Place


## Cluster 4

In [40]:
van_merged.loc[van_merged['Cluster Labels'] == 4, van_merged.columns[[2] + list(range(4, van_merged.shape[1]))]]

,Neighbourhoods,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Vancouver (NE Downtown / Harbour Centre / Gast...,49.2788,-123.1139,4,Hotel,Restaurant,Bakery,Dessert Shop,Sandwich Place,Coffee Shop,Park,Japanese Restaurant,Breakfast Spot,Taco Place
15,Vancouver (Waterfront / Coal Harbour / Canada ...,49.2866,-123.1158,4,Hotel,Restaurant,Dessert Shop,Café,Sandwich Place,Breakfast Spot,Coffee Shop,Plaza,Concert Hall,Gym
16,Vancouver (South West End),49.2833,-123.1298,4,Hotel,Dessert Shop,Bakery,Restaurant,Japanese Restaurant,Park,Seafood Restaurant,Sandwich Place,Ramen Restaurant,Coffee Shop
17,Vancouver (North West End / Stanley Park),49.2990,-123.1408,4,Coffee Shop,Scenic Lookout,Seafood Restaurant,Park,Sculpture Garden,Restaurant,Ramen Restaurant,Trail,Beach,Hotel
28,Vancouver (SW Downtown),49.2767,-123.1300,4,Hotel,Seafood Restaurant,Bakery,Restaurant,Park,Dessert Shop,Italian Restaurant,Trail,Coffee Shop,French Restaurant
42,Vancouver (Bentall Centre),49.2935,-123.1162,4,Hotel,Coffee Shop,Dessert Shop,Sandwich Place,Restaurant,Café,Plaza,Park,Ramen Restaurant,Bookstore
43,Vancouver (Pacific Centre),49.2819,-123.1190,4,Hotel,Restaurant,Dessert Shop,Sandwich Place,Bakery,Coffee Shop,Italian Restaurant,Japanese Restaurant,Café,Ice Cream Shop


In [41]:
van_merged.loc[van_merged['Cluster Labels'] == 5, van_merged.columns[[2] + list(range(4, van_merged.shape[1]))]]

,Neighbourhoods,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
29,North Vancouver Outer East,49.3678,-122.9278,5,Trail,Ski Chairlift,Ski Lodge,Outdoors & Recreation,Restaurant,Cafeteria,Ski Area,Scenic Lookout,Filipino Restaurant,Financial or Legal Service
37,North Vancouver Northwest,49.3775,-123.0862,5,Trail,Scenic Lookout,Ski Chairlift,Coffee Shop,Mountain,Ski Chalet,Restaurant,Fast Food Restaurant,Dessert Shop,Sporting Goods Shop
